<a href="https://colab.research.google.com/github/corypham/AggiePulse/blob/main/process_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Post Processing for Tennis Court & Ball Detection

This notebook leverages Google Colab's online GPUs for CourtCheck's post processing. Pretrained weights are referenced in the **last cell** for both Court and Ball detection models. Please run each cell **chronologically** and change your input & output paths in **mp4 format**. For any questions, please contact corypham1@gmail.com

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Basic dependencies
!pip install numpy opencv-python torch torchvision tqdm scipy matplotlib

# Scene detection
!pip install scenedetect

# CatBoost for bounce detection
!pip install catboost

# For visualization
!pip install opencv-python-headless

!pip install CubicSpline

# If you need CUDA support (usually pre-installed in Colab)
import torch
print(torch.cuda.is_available())  # Should print True

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
import sys
import os
import warnings
import logging
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch
import time
from scipy.spatial import distance
from itertools import groupby
from tqdm import tqdm
from collections import deque
import catboost as ctb
import pandas as pd
import torch.nn.functional as F
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors import ContentDetector
from scipy.interpolate import CubicSpline
from sympy import Line
from scipy.spatial import distance
from scipy import signal
import matplotlib.pyplot as plt
from sympy.geometry.point import Point2D




In [ ]:
## Tracknet script

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, pad=1, stride=1, bias=True):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=pad, bias=bias),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.block(x)

class BallTrackerNet(nn.Module):
    def __init__(self, input_channels=3, out_channels=14):
        super().__init__()
        self.out_channels = out_channels
        self.input_channels = input_channels

        self.conv1 = ConvBlock(in_channels=self.input_channels, out_channels=64)
        self.conv2 = ConvBlock(in_channels=64, out_channels=64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = ConvBlock(in_channels=64, out_channels=128)
        self.conv4 = ConvBlock(in_channels=128, out_channels=128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = ConvBlock(in_channels=128, out_channels=256)
        self.conv6 = ConvBlock(in_channels=256, out_channels=256)
        self.conv7 = ConvBlock(in_channels=256, out_channels=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv8 = ConvBlock(in_channels=256, out_channels=512)
        self.conv9 = ConvBlock(in_channels=512, out_channels=512)
        self.conv10 = ConvBlock(in_channels=512, out_channels=512)
        self.ups1 = nn.Upsample(scale_factor=2)
        self.conv11 = ConvBlock(in_channels=512, out_channels=256)
        self.conv12 = ConvBlock(in_channels=256, out_channels=256)
        self.conv13 = ConvBlock(in_channels=256, out_channels=256)
        self.ups2 = nn.Upsample(scale_factor=2)
        self.conv14 = ConvBlock(in_channels=256, out_channels=128)
        self.conv15 = ConvBlock(in_channels=128, out_channels=128)
        self.ups3 = nn.Upsample(scale_factor=2)
        self.conv16 = ConvBlock(in_channels=128, out_channels=64)
        self.conv17 = ConvBlock(in_channels=64, out_channels=64)
        self.conv18 = ConvBlock(in_channels=64, out_channels=self.out_channels)

        self._init_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.ups1(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.ups2(x)
        x = self.conv14(x)
        x = self.conv15(x)
        x = self.ups3(x)
        x = self.conv16(x)
        x = self.conv17(x)
        x = self.conv18(x)
        return x

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.uniform_(module.weight, -0.05, 0.05)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)

            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)


In [ ]:
# Utility Script


def scene_detect(path_video):
    """
    Split video to disjoint fragments based on color histograms
    using PySceneDetect.
    """
    video_manager = VideoManager([path_video])
    stats_manager = StatsManager()
    scene_manager = SceneManager(stats_manager)
    scene_manager.add_detector(ContentDetector())
    base_timecode = video_manager.get_base_timecode()

    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list(base_timecode)

    if not scene_list:
        scene_list = [
            (video_manager.get_base_timecode(), video_manager.get_current_timecode())
        ]
    scenes = [[x[0].frame_num, x[1].frame_num] for x in scene_list]
    return scenes


def build_heatmap_court_background_black():
    """
    Black background with thick white court lines.
    """
    raw_court = CourtReference().build_court_reference()
    raw_court = cv2.dilate(raw_court, np.ones((10, 10), dtype=np.uint8))

    background = np.zeros_like(raw_court, dtype=np.uint8)
    background[raw_court == 1] = 255  # white lines
    return cv2.cvtColor(background, cv2.COLOR_GRAY2BGR)


def build_custom_colormap_black_purple_red_green_yellow():
    """
    Creates a custom color map (256 x 1 x 3) with 5 anchors:
      0   -> black
      64  -> purple
      128 -> red
      192 -> green
      255 -> bright yellow
    This ensures that 'no data' (0) stays black, and high frequency = yellow.
    """
    anchors = [
        (0, (0, 0, 0)),  # black
        (64, (128, 0, 128)),  # purple
        (128, (0, 0, 255)),  # red
        (192, (0, 255, 0)),  # green
        (255, (0, 255, 255)),  # bright yellow
    ]
    ctable = np.zeros((256, 1, 3), dtype=np.uint8)

    def lerp_color(c1, c2, t):
        return (
            int(c1[0] + (c2[0] - c1[0]) * t),
            int(c1[1] + (c2[1] - c1[1]) * t),
            int(c1[2] + (c2[2] - c1[2]) * t),
        )

    for i in range(len(anchors) - 1):
        start_idx, start_col = anchors[i]
        end_idx, end_col = anchors[i + 1]
        for x in range(start_idx, end_idx + 1):
            if end_idx == start_idx:
                t = 0
            else:
                t = (x - start_idx) / float(end_idx - start_idx)
            ctable[x, 0] = lerp_color(start_col, end_col, t)

    return ctable


def generate_minimap_heatmaps(
    homography_matrices,
    ball_track,
    bounces,
    persons_top,
    persons_bottom,
    output_bounce_heatmap,
    output_player_heatmap,
    blur_ksize=41,
    alpha=0.5,
):
    """
    1) For ball bounces, draw bigger bright‐yellow circles (radius=8) with red outline.
    2) For player positions, accumulate + blur, then apply a custom colormap:
       black->purple->red->green->yellow, so zero=black, max=yellow.
    3) The background stays black with white lines (where no data is present).
    """

    # (A) Build black court background
    court_img = build_heatmap_court_background_black()
    Hc, Wc = court_img.shape[:2]
    n_frames = len(homography_matrices)

    # (B) Bounces => direct drawing
    bounce_overlay = court_img.copy()
    for i in range(n_frames):
        if i not in bounces:
            continue
        bx, by = ball_track[i]
        inv_mat = homography_matrices[i]
        if bx is None or inv_mat is None:
            continue

        pt = np.array([[[bx, by]]], dtype=np.float32)
        mapped = cv2.perspectiveTransform(pt, inv_mat)
        xx, yy = int(mapped[0, 0, 0]), int(mapped[0, 0, 1])
        if 0 <= xx < Wc and 0 <= yy < Hc:
            # Increase radius from 5 to 8 for bigger circles
            cv2.circle(bounce_overlay, (xx, yy), 12, (0, 255, 255), -1)  # fill (yellow)
            cv2.circle(bounce_overlay, (xx, yy), 12, (0, 0, 255), 2)  # outline (red)

    # (C) Players => aggregator -> blur -> custom colormap
    player_acc = np.zeros((Hc, Wc), dtype=np.float32)
    for i in range(n_frames):
        inv_mat = homography_matrices[i]
        if inv_mat is None:
            continue

        # top
        for bbox, center_pt in persons_top[i]:
            if bbox is not None and len(bbox) == 4:
                cx, cy = center_pt
                pt = np.array([[[cx, cy]]], dtype=np.float32)
                mapped = cv2.perspectiveTransform(pt, inv_mat)
                xx, yy = int(mapped[0, 0, 0]), int(mapped[0, 0, 1])
                if 0 <= xx < Wc and 0 <= yy < Hc:
                    cv2.circle(player_acc, (xx, yy), 10, 1.0, -1)

        # bottom
        for bbox, center_pt in persons_bottom[i]:
            if bbox is not None and len(bbox) == 4:
                cx, cy = center_pt
                pt = np.array([[[cx, cy]]], dtype=np.float32)
                mapped = cv2.perspectiveTransform(pt, inv_mat)
                xx, yy = int(mapped[0, 0, 0]), int(mapped[0, 0, 1])
                if 0 <= xx < Wc and 0 <= yy < Hc:
                    cv2.circle(player_acc, (xx, yy), 10, 1.0, -1)

    # blur => smoother distribution
    player_blurred = cv2.GaussianBlur(player_acc, (blur_ksize, blur_ksize), 0)

    # (D) Convert blurred player data -> custom colormap
    mx = player_blurred.max()
    if mx < 1e-8:
        # no data
        player_overlay = court_img.copy()
    else:
        norm = (player_blurred / mx * 255).astype(np.uint8)
        custom_cmap = build_custom_colormap_black_purple_red_green_yellow()
        player_heat = cv2.applyColorMap(norm, custom_cmap)
        player_overlay = cv2.addWeighted(court_img, 1.0, player_heat, alpha, 0.0)

    # (E) Save
    cv2.imwrite(output_bounce_heatmap, bounce_overlay)
    cv2.imwrite(output_player_heatmap, player_overlay)
    print(f"Saved bounce heatmap to: {output_bounce_heatmap}")
    print(f"Saved player heatmap to: {output_player_heatmap}")


In [ ]:
## Ball Detection

class BallDetector:
    def __init__(self, path_model=None, device='cuda'):
        self.model = BallTrackerNet(input_channels=9, out_channels=256)
        self.device = device
        if path_model:
            self.model.load_state_dict(torch.load(path_model, map_location=device))
            self.model = self.model.to(device)
            self.model.eval()
        self.width = 640
        self.height = 360

    def infer_model(self, frames):
        """ Run pretrained model on a consecutive list of frames
        :params
            frames: list of consecutive video frames
        :return
            ball_track: list of detected ball points
        """
        ball_track = [(None, None)]*2
        prev_pred = [None, None]
        for num in tqdm(range(2, len(frames))):
            img = cv2.resize(frames[num], (self.width, self.height))
            img_prev = cv2.resize(frames[num-1], (self.width, self.height))
            img_preprev = cv2.resize(frames[num-2], (self.width, self.height))
            imgs = np.concatenate((img, img_prev, img_preprev), axis=2)
            imgs = imgs.astype(np.float32)/255.0
            imgs = np.rollaxis(imgs, 2, 0)
            inp = np.expand_dims(imgs, axis=0)

            out = self.model(torch.from_numpy(inp).float().to(self.device))
            output = out.argmax(dim=1).detach().cpu().numpy()
            x_pred, y_pred = self.postprocess(output, prev_pred)
            prev_pred = [x_pred, y_pred]
            ball_track.append((x_pred, y_pred))
        return ball_track

    def postprocess(self, feature_map, prev_pred, scale=2, max_dist=80):
        """
        :params
            feature_map: feature map with shape (1,360,640)
            prev_pred: [x,y] coordinates of ball prediction from previous frame
            scale: scale for conversion to original shape (720,1280)
            max_dist: maximum distance from previous ball detection to remove outliers
        :return
            x,y ball coordinates
        """
        feature_map *= 255
        feature_map = feature_map.reshape((self.height, self.width))
        feature_map = feature_map.astype(np.uint8)
        ret, heatmap = cv2.threshold(feature_map, 127, 255, cv2.THRESH_BINARY)
        circles = cv2.HoughCircles(heatmap, cv2.HOUGH_GRADIENT, dp=1, minDist=1, param1=50, param2=2, minRadius=2,
                                   maxRadius=7)
        x, y = None, None
        if circles is not None:
            if prev_pred[0]:
                for i in range(len(circles[0])):
                    x_temp = circles[0][i][0]*scale
                    y_temp = circles[0][i][1]*scale
                    dist = distance.euclidean((x_temp, y_temp), prev_pred)
                    if dist < max_dist:
                        x, y = x_temp, y_temp
                        break
            else:
                x = circles[0][0][0]*scale
                y = circles[0][0][1]*scale
        return x, y

In [ ]:
## Court Reference


class CourtReference:
    """
    Court reference model
    """
    def __init__(self):
        self.baseline_top = ((286, 561), (1379, 561))
        self.baseline_bottom = ((286, 2935), (1379, 2935))
        self.net = ((286, 1748), (1379, 1748))
        self.left_court_line = ((286, 561), (286, 2935))
        self.right_court_line = ((1379, 561), (1379, 2935))
        self.left_inner_line = ((423, 561), (423, 2935))
        self.right_inner_line = ((1242, 561), (1242, 2935))
        self.middle_line = ((832, 1110), (832, 2386))
        self.top_inner_line = ((423, 1110), (1242, 1110))
        self.bottom_inner_line = ((423, 2386), (1242, 2386))
        self.top_extra_part = (832.5, 580)
        self.bottom_extra_part = (832.5, 2910)

        self.key_points = [*self.baseline_top, *self.baseline_bottom,
                          *self.left_inner_line, *self.right_inner_line,
                          *self.top_inner_line, *self.bottom_inner_line,
                          *self.middle_line]

        self.border_points = [*self.baseline_top, *self.baseline_bottom[::-1]]

        self.court_conf = {1: [*self.baseline_top, *self.baseline_bottom],
                           2: [self.left_inner_line[0], self.right_inner_line[0], self.left_inner_line[1],
                               self.right_inner_line[1]],
                           3: [self.left_inner_line[0], self.right_court_line[0], self.left_inner_line[1],
                               self.right_court_line[1]],
                           4: [self.left_court_line[0], self.right_inner_line[0], self.left_court_line[1],
                               self.right_inner_line[1]],
                           5: [*self.top_inner_line, *self.bottom_inner_line],
                           6: [*self.top_inner_line, self.left_inner_line[1], self.right_inner_line[1]],
                           7: [self.left_inner_line[0], self.right_inner_line[0], *self.bottom_inner_line],
                           8: [self.right_inner_line[0], self.right_court_line[0], self.right_inner_line[1],
                               self.right_court_line[1]],
                           9: [self.left_court_line[0], self.left_inner_line[0], self.left_court_line[1],
                               self.left_inner_line[1]],
                           10: [self.top_inner_line[0], self.middle_line[0], self.bottom_inner_line[0],
                                self.middle_line[1]],
                           11: [self.middle_line[0], self.top_inner_line[1], self.middle_line[1],
                                self.bottom_inner_line[1]],
                           12: [*self.bottom_inner_line, self.left_inner_line[1], self.right_inner_line[1]]}
        self.line_width = 1
        self.court_width = 1117
        self.court_height = 2408
        self.top_bottom_border = 549
        self.right_left_border = 274
        self.court_total_width = self.court_width + self.right_left_border * 2
        self.court_total_height = self.court_height + self.top_bottom_border * 2
        self.court = self.build_court_reference()

        # self.court = cv2.cvtColor(cv2.imread('court_configurations/court_reference.png'), cv2.COLOR_BGR2GRAY)

    def build_court_reference(self):
        """
        Create court reference image using the lines positions
        """
        court = np.zeros((self.court_height + 2 * self.top_bottom_border, self.court_width + 2 * self.right_left_border), dtype=np.uint8)
        cv2.line(court, *self.baseline_top, 1, self.line_width)
        cv2.line(court, *self.baseline_bottom, 1, self.line_width)
        cv2.line(court, *self.net, 1, self.line_width)
        cv2.line(court, *self.top_inner_line, 1, self.line_width)
        cv2.line(court, *self.bottom_inner_line, 1, self.line_width)
        cv2.line(court, *self.left_court_line, 1, self.line_width)
        cv2.line(court, *self.right_court_line, 1, self.line_width)
        cv2.line(court, *self.left_inner_line, 1, self.line_width)
        cv2.line(court, *self.right_inner_line, 1, self.line_width)
        cv2.line(court, *self.middle_line, 1, self.line_width)
        court = cv2.dilate(court, np.ones((5, 5), dtype=np.uint8))
        # court = cv2.dilate(court, np.ones((7, 7), dtype=np.uint8))
        # plt.imsave('court_configurations/court_reference.png', court, cmap='gray')
        # self.court = court
        return court

    def get_important_lines(self):
        """
        Returns all lines of the court
        """
        lines = [*self.baseline_top, *self.baseline_bottom, *self.net, *self.left_court_line, *self.right_court_line,
                 *self.left_inner_line, *self.right_inner_line, *self.middle_line,
                 *self.top_inner_line, *self.bottom_inner_line]
        return lines

    def get_extra_parts(self):
        parts = [self.top_extra_part, self.bottom_extra_part]
        return parts

    def save_all_court_configurations(self):
        """
        Create all configurations of 4 points on court reference
        """
        for i, conf in self.court_conf.items():
            c = cv2.cvtColor(255 - self.court, cv2.COLOR_GRAY2BGR)
            for p in conf:
                c = cv2.circle(c, p, 15, (0, 0, 255), 30)
            cv2.imwrite(f'court_configurations/court_conf_{i}.png', c)

    def get_court_mask(self, mask_type=0):
        """
        Get mask of the court
        """
        mask = np.ones_like(self.court)
        if mask_type == 1:  # Bottom half court
            # mask[:self.net[0][1] - 1000, :] = 0
            mask[:self.net[0][1], :] = 0
        elif mask_type == 2:  # Top half court
            mask[self.net[0][1]:, :] = 0
        elif mask_type == 3: # court without margins
            mask[:self.baseline_top[0][1], :] = 0
            mask[self.baseline_bottom[0][1]:, :] = 0
            mask[:, :self.left_court_line[0][0]] = 0
            mask[:, self.right_court_line[0][0]:] = 0
        return mask


if __name__ == '__main__':
    c = CourtReference()
    c.build_court_reference()


In [ ]:
## Homography

court_ref = CourtReference()
refer_kps = np.array(court_ref.key_points, dtype=np.float32).reshape((-1, 1, 2))

court_conf_ind = {}
for i in range(len(court_ref.court_conf)):
    conf = court_ref.court_conf[i+1]
    inds = []
    for j in range(4):
        inds.append(court_ref.key_points.index(conf[j]))
    court_conf_ind[i+1] = inds

def get_trans_matrix(points):
    """
    Determine the best homography matrix from court points
    """
    matrix_trans = None
    dist_max = np.Inf
    for conf_ind in range(1, 13):
        conf = court_ref.court_conf[conf_ind]

        inds = court_conf_ind[conf_ind]
        inters = [points[inds[0]], points[inds[1]], points[inds[2]], points[inds[3]]]
        if None not in inters:
            matrix, _ = cv2.findHomography(np.float32(conf), np.float32(inters), method=0)
            trans_kps = cv2.perspectiveTransform(refer_kps, matrix).squeeze(1)
            dists = []
            for i in range(12):
                if i not in inds and points[i] is not None:
                    dists.append(distance.euclidean(points[i], trans_kps[i]))
            dist_median = np.mean(dists)
            if dist_median < dist_max:
                matrix_trans = matrix
                dist_max = dist_median
    return matrix_trans


In [ ]:
## Court Detection

class CourtDetectorNet():
    def __init__(self, path_model=None,  device='cuda'):
        self.model = BallTrackerNet(out_channels=15)
        self.device = device
        if path_model:
            self.model.load_state_dict(torch.load(path_model, map_location=device))
            self.model = self.model.to(device)
            self.model.eval()

    def infer_model(self, frames):
        output_width = 640
        output_height = 360
        scale = 2

        kps_res = []
        matrixes_res = []
        for num_frame, image in enumerate(tqdm(frames)):
            img = cv2.resize(image, (output_width, output_height))
            inp = (img.astype(np.float32) / 255.)
            inp = torch.tensor(np.rollaxis(inp, 2, 0))
            inp = inp.unsqueeze(0)

            out = self.model(inp.float().to(self.device))[0]
            pred = F.sigmoid(out).detach().cpu().numpy()

            points = []
            for kps_num in range(14):
                heatmap = (pred[kps_num]*255).astype(np.uint8)
                ret, heatmap = cv2.threshold(heatmap, 170, 255, cv2.THRESH_BINARY)
                circles = cv2.HoughCircles(heatmap, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=2,
                                           minRadius=10, maxRadius=25)
                if circles is not None:
                    x_pred = circles[0][0][0]*scale
                    y_pred = circles[0][0][1]*scale
                    if kps_num not in [8, 12, 9]:
                        x_pred, y_pred = refine_kps(image, int(y_pred), int(x_pred), crop_size=40)
                    points.append((x_pred, y_pred))
                else:
                    points.append(None)

            matrix_trans = get_trans_matrix(points)
            points = None
            if matrix_trans is not None:
                points = cv2.perspectiveTransform(refer_kps, matrix_trans)
                matrix_trans = cv2.invert(matrix_trans)[1]
            kps_res.append(points)
            matrixes_res.append(matrix_trans)

        return matrixes_res, kps_res

In [ ]:
## Postprocess

def line_intersection(line1, line2):
    """
    Find 2 lines intersection point
    """
    l1 = Line((line1[0], line1[1]), (line1[2], line1[3]))
    l2 = Line((line2[0], line2[1]), (line2[2], line2[3]))

    intersection = l1.intersection(l2)
    point = None
    if len(intersection) > 0:
        if isinstance(intersection[0], Point2D):
            point = intersection[0].coordinates
    return point

def refine_kps(img, x_ct, y_ct, crop_size=40):
    refined_x_ct, refined_y_ct = x_ct, y_ct

    img_height, img_width = img.shape[:2]
    x_min = max(x_ct-crop_size, 0)
    x_max = min(img_height, x_ct+crop_size)
    y_min = max(y_ct-crop_size, 0)
    y_max = min(img_width, y_ct+crop_size)

    img_crop = img[x_min:x_max, y_min:y_max]
    lines = detect_lines(img_crop)
    # print('lines = ', lines)

    if len(lines) > 1:
        lines = merge_lines(lines)
        if len(lines) == 2:
            inters = line_intersection(lines[0], lines[1])
            if inters:
                new_x_ct = int(inters[1])
                new_y_ct = int(inters[0])
                if new_x_ct > 0 and new_x_ct < img_crop.shape[0] and new_y_ct > 0 and new_y_ct < img_crop.shape[1]:
                    refined_x_ct = x_min + new_x_ct
                    refined_y_ct = y_min + new_y_ct
    return refined_y_ct, refined_x_ct

def detect_lines(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY)[1]
    lines = cv2.HoughLinesP(gray, 1, np.pi / 180, 30, minLineLength=10, maxLineGap=30)
    lines = np.squeeze(lines)
    if len(lines.shape) > 0:
        if len(lines) == 4 and not isinstance(lines[0], np.ndarray):
            lines = [lines]
    else:
        lines = []
    return lines

def merge_lines(lines):
    lines = sorted(lines, key=lambda item: item[0])
    mask = [True] * len(lines)
    new_lines = []

    for i, line in enumerate(lines):
        if mask[i]:
            for j, s_line in enumerate(lines[i + 1:]):
                if mask[i + j + 1]:
                    x1, y1, x2, y2 = line
                    x3, y3, x4, y4 = s_line
                    dist1 = distance.euclidean((x1, y1), (x3, y3))
                    dist2 = distance.euclidean((x2, y2), (x4, y4))
                    if dist1 < 20 and dist2 < 20:
                        line = np.array([int((x1+x3)/2), int((y1+y3)/2), int((x2+x4)/2), int((y2+y4)/2)])
                        mask[i + j + 1] = False
            new_lines.append(line)
    return new_lines


In [ ]:
## Person Detection

class PersonDetector():
    def __init__(self, dtype=torch.FloatTensor):
        self.detection_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        self.detection_model = self.detection_model.to(dtype)
        self.detection_model.eval()
        self.dtype = dtype
        self.court_ref = CourtReference()
        self.ref_top_court = self.court_ref.get_court_mask(2)
        self.ref_bottom_court = self.court_ref.get_court_mask(1)
        self.point_person_top = None
        self.point_person_bottom = None
        self.counter_top = 0
        self.counter_bottom = 0


    def detect(self, image, person_min_score=0.85):
        PERSON_LABEL = 1
        frame_tensor = image.transpose((2, 0, 1)) / 255
        frame_tensor = torch.from_numpy(frame_tensor).unsqueeze(0).float().to(self.dtype)

        with torch.no_grad():
            preds = self.detection_model(frame_tensor)

        persons_boxes = []
        probs = []
        for box, label, score in zip(preds[0]['boxes'][:], preds[0]['labels'], preds[0]['scores']):
            if label == PERSON_LABEL and score > person_min_score:
                persons_boxes.append(box.detach().cpu().numpy())
                probs.append(score.detach().cpu().numpy())
        return persons_boxes, probs

    def detect_top_and_bottom_players(self, image, inv_matrix, filter_players=False):
        matrix = cv2.invert(inv_matrix)[1]
        mask_top_court = cv2.warpPerspective(self.ref_top_court, matrix, image.shape[1::-1])
        mask_bottom_court = cv2.warpPerspective(self.ref_bottom_court, matrix, image.shape[1::-1])
        person_bboxes_top, person_bboxes_bottom = [], []

        bboxes, probs = self.detect(image, person_min_score=0.85)
        if len(bboxes) > 0:
            person_points = [[int((bbox[2] + bbox[0]) / 2), int(bbox[3])] for bbox in bboxes]
            person_bboxes = list(zip(bboxes, person_points))

            person_bboxes_top = [pt for pt in person_bboxes if mask_top_court[pt[1][1]-1, pt[1][0]] == 1]
            person_bboxes_bottom = [pt for pt in person_bboxes if mask_bottom_court[pt[1][1] - 1, pt[1][0]] == 1]

            if filter_players:
                person_bboxes_top, person_bboxes_bottom = self.filter_players(person_bboxes_top, person_bboxes_bottom,
                                                                              matrix)
        return person_bboxes_top, person_bboxes_bottom

    def filter_players(self, person_bboxes_top, person_bboxes_bottom, matrix):
        """
        Leave one person at the top and bottom of the tennis court
        """
        refer_kps = np.array(self.court_ref.key_points[12:], dtype=np.float32).reshape((-1, 1, 2))
        trans_kps = cv2.perspectiveTransform(refer_kps, matrix)
        center_top_court = trans_kps[0][0]
        center_bottom_court = trans_kps[1][0]
        if len(person_bboxes_top) > 1:
            dists = [distance.euclidean(x[1], center_top_court) for x in person_bboxes_top]
            ind = dists.index(min(dists))
            person_bboxes_top = [person_bboxes_top[ind]]
        if len(person_bboxes_bottom) > 1:
            dists = [distance.euclidean(x[1], center_bottom_court) for x in person_bboxes_bottom]
            ind = dists.index(min(dists))
            person_bboxes_bottom = [person_bboxes_bottom[ind]]
        return person_bboxes_top, person_bboxes_bottom

    def track_players(self, frames, matrix_all, filter_players=False):
        persons_top = []
        persons_bottom = []
        min_len = min(len(frames), len(matrix_all))
        for num_frame in tqdm(range(min_len)):
            img = frames[num_frame]
            if matrix_all[num_frame] is not None:
                inv_matrix = matrix_all[num_frame]
                person_top, person_bottom = self.detect_top_and_bottom_players(img, inv_matrix, filter_players)
            else:
                person_top, person_bottom = [], []
            persons_top.append(person_top)
            persons_bottom.append(person_bottom)
        return persons_top, persons_bottom




In [ ]:
## Bounce Detection

class BounceDetector:
    def __init__(self, path_model=None):
        self.model = ctb.CatBoostRegressor()
        self.threshold = 0.45
        if path_model:
            self.load_model(path_model)

    def load_model(self, path_model):
        self.model.load_model(path_model)

    def prepare_features(self, x_ball, y_ball):
        labels = pd.DataFrame({'frame': range(len(x_ball)), 'x-coordinate': x_ball, 'y-coordinate': y_ball})

        num = 3
        eps = 1e-15
        for i in range(1, num):
            labels['x_lag_{}'.format(i)] = labels['x-coordinate'].shift(i)
            labels['x_lag_inv_{}'.format(i)] = labels['x-coordinate'].shift(-i)
            labels['y_lag_{}'.format(i)] = labels['y-coordinate'].shift(i)
            labels['y_lag_inv_{}'.format(i)] = labels['y-coordinate'].shift(-i)
            labels['x_diff_{}'.format(i)] = abs(labels['x_lag_{}'.format(i)] - labels['x-coordinate'])
            labels['y_diff_{}'.format(i)] = labels['y_lag_{}'.format(i)] - labels['y-coordinate']
            labels['x_diff_inv_{}'.format(i)] = abs(labels['x_lag_inv_{}'.format(i)] - labels['x-coordinate'])
            labels['y_diff_inv_{}'.format(i)] = labels['y_lag_inv_{}'.format(i)] - labels['y-coordinate']
            labels['x_div_{}'.format(i)] = abs(labels['x_diff_{}'.format(i)]/(labels['x_diff_inv_{}'.format(i)] + eps))
            labels['y_div_{}'.format(i)] = labels['y_diff_{}'.format(i)]/(labels['y_diff_inv_{}'.format(i)] + eps)

        for i in range(1, num):
            labels = labels[labels['x_lag_{}'.format(i)].notna()]
            labels = labels[labels['x_lag_inv_{}'.format(i)].notna()]
        labels = labels[labels['x-coordinate'].notna()]

        colnames_x = ['x_diff_{}'.format(i) for i in range(1, num)] + \
                     ['x_diff_inv_{}'.format(i) for i in range(1, num)] + \
                     ['x_div_{}'.format(i) for i in range(1, num)]
        colnames_y = ['y_diff_{}'.format(i) for i in range(1, num)] + \
                     ['y_diff_inv_{}'.format(i) for i in range(1, num)] + \
                     ['y_div_{}'.format(i) for i in range(1, num)]
        colnames = colnames_x + colnames_y

        features = labels[colnames]
        return features, list(labels['frame'])

    def predict(self, x_ball, y_ball, smooth=True):
        if smooth:
            x_ball, y_ball = self.smooth_predictions(x_ball, y_ball)
        features, num_frames = self.prepare_features(x_ball, y_ball)
        preds = self.model.predict(features)
        ind_bounce = np.where(preds > self.threshold)[0]
        if len(ind_bounce) > 0:
            ind_bounce = self.postprocess(ind_bounce, preds)
        frames_bounce = [num_frames[x] for x in ind_bounce]
        return set(frames_bounce)

    def smooth_predictions(self, x_ball, y_ball):
        is_none = [int(x is None) for x in x_ball]
        interp = 5
        counter = 0
        for num in range(interp, len(x_ball)-1):
            if not x_ball[num] and sum(is_none[num-interp:num]) == 0 and counter < 3:
                x_ext, y_ext = self.extrapolate(x_ball[num-interp:num], y_ball[num-interp:num])
                x_ball[num] = x_ext
                y_ball[num] = y_ext
                is_none[num] = 0
                if x_ball[num+1]:
                    dist = distance.euclidean((x_ext, y_ext), (x_ball[num+1], y_ball[num+1]))
                    if dist > 80:
                        x_ball[num+1], y_ball[num+1], is_none[num+1] = None, None, 1
                counter += 1
            else:
                counter = 0
        return x_ball, y_ball

    def extrapolate(self, x_coords, y_coords):
        xs = list(range(len(x_coords)))
        func_x = CubicSpline(xs, x_coords, bc_type='natural')
        x_ext = func_x(len(x_coords))
        func_y = CubicSpline(xs, y_coords, bc_type='natural')
        y_ext = func_y(len(x_coords))
        return float(x_ext), float(y_ext)

    def postprocess(self, ind_bounce, preds):
        ind_bounce_filtered = [ind_bounce[0]]
        for i in range(1, len(ind_bounce)):
            if (ind_bounce[i] - ind_bounce[i-1]) != 1:
                cur_ind = ind_bounce[i]
                ind_bounce_filtered.append(cur_ind)
            elif preds[ind_bounce[i]] > preds[ind_bounce[i-1]]:
                ind_bounce_filtered[-1] = ind_bounce[i]
        return ind_bounce_filtered

In [ ]:
keypoint_names = [
    "BTL",
    "BTR",
    "BBL",
    "BBR",
    "BTLI",
    "BBLI",
    "BTRI",
    "BBRI",
    "ITL",
    "ITR",
    "IBL",
    "IBR",
    "ITM",
    "IBM",
]

court_lines = [
    ("BTL", "BTLI"),
    ("BTLI", "BTRI"),
    ("BTRI", "BTR"),
    ("BTL", "BBL"),
    ("BTR", "BBR"),
    ("BBL", "BBLI"),
    ("BBLI", "BBRI"),
    ("BBLI", "IBL"),
    ("BBRI", "IBR"),
    ("BBRI", "BBR"),
    ("BTLI", "ITL"),
    ("BTRI", "ITR"),
    ("ITL", "ITM"),
    ("ITM", "IBM"),
    ("ITL", "IBL"),
    ("ITR", "IBR"),
    ("IBL", "IBM"),
    ("IBM", "IBR"),
    ("ITM", "ITR"),
]


def ensure_720p(input_path, intermediate_path):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    print(f"Original input: {width}x{height}, fps={fps:.2f}")
    if (width != 1280) or (height != 720):
        print(f"Resizing from ({width}x{height}) to (1280x720) -> {intermediate_path}")
        cap_in = cv2.VideoCapture(input_path)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(intermediate_path, fourcc, fps, (1280, 720))

        while True:
            ret, frame = cap_in.read()
            if not ret:
                break
            frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_AREA)
            out.write(frame)

        cap_in.release()
        out.release()
        print(f"Finished writing intermediate: {intermediate_path}")
        return intermediate_path
    else:
        print("Video is already 1280x720; using input directly.")
        return input_path


def read_video(path_video):
    cap = cv2.VideoCapture(path_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames, fps


def get_court_img():
    """Build a 720p-like minimap with white lines on black background."""
    court_ref = CourtReference()
    court = court_ref.build_court_reference()
    court = cv2.dilate(court, np.ones((10, 10), dtype=np.uint8))
    court_img = (np.stack((court, court, court), axis=2) * 255).astype(np.uint8)
    return court_img


def draw_court_keypoints_and_lines(frame, kps, frame_width, frame_height):
    """
    Draw tennis court lines (green) and keypoints (red) on 'frame'.
    """
    for start_name, end_name in court_lines:
        try:
            s_idx = keypoint_names.index(start_name)
            e_idx = keypoint_names.index(end_name)
            if kps[s_idx] is None or kps[e_idx] is None:
                continue
            x1 = int(kps[s_idx][0, 0] * frame_width / 1280)
            y1 = int(kps[s_idx][0, 1] * frame_height / 720)
            x2 = int(kps[e_idx][0, 0] * frame_width / 1280)
            y2 = int(kps[e_idx][0, 1] * frame_height / 720)
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        except ValueError:
            pass

    # Keypoints
    for i, pt in enumerate(kps):
        if pt is None:
            continue
        x = int(pt[0, 0] * frame_width / 1280)
        y = int(pt[0, 1] * frame_height / 720)
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        label = keypoint_names[i]
        (tw, th), base = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
        cv2.rectangle(
            frame, (x - 5, y - th - 5), (x - 5 + tw, y - 5), (255, 255, 255), -1
        )
        cv2.putText(
            frame, label, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1
        )


def main(
    frames,
    scenes,
    bounces,
    ball_track,
    homography_matrices,
    kps_court,
    persons_top,
    persons_bottom,
    draw_trace=True,
    trace=7,
):
    imgs_res = []
    minimap_frames = []  # New list to store minimap frames
    width_minimap = 166
    height_minimap = 350

    light_blue = (255, 255, 0)
    bounce_color = (0, 255, 255)
    box_color = (255, 0, 0)

    for start_idx, end_idx in scenes:
        valid_count = sum(
            1
            for idx in range(start_idx, end_idx)
            if homography_matrices[idx] is not None
        )
        scene_len = end_idx - start_idx
        scene_rate = valid_count / (scene_len + 1e-15)

        if scene_rate > 0.5:
            for i in range(start_idx, end_idx):
                court_img = get_court_img()
                img_res = frames[i]
                inv_mat = homography_matrices[i]

                # ball
                if ball_track[i][0]:
                    bx = int(ball_track[i][0])
                    by = int(ball_track[i][1])
                    if draw_trace:
                        for j in range(trace):
                            idx = i - j
                            if idx < 0:
                                break
                            if ball_track[idx][0]:
                                px, py = ball_track[idx]
                                alpha = 1.0 - (j / trace)
                                color_fade = tuple(int(c * alpha) for c in light_blue)
                                cv2.circle(
                                    img_res, (int(px), int(py)), 3, color_fade, -1
                                )
                    else:
                        cv2.circle(img_res, (bx, by), 5, light_blue, -1)
                        cv2.putText(
                            img_res,
                            "ball",
                            (bx + 8, by + 8),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.8,
                            light_blue,
                            2,
                        )

                # ball trace on minimap
                if inv_mat is not None and ball_track[i][0]:
                    for j in range(trace):
                        idx = i - j
                        if idx < 0:
                            break
                        if ball_track[idx][0]:
                            px, py = ball_track[idx]
                            arr = cv2.perspectiveTransform(
                                np.array([[[px, py]]], dtype=np.float32), inv_mat
                            )
                            alpha_ = 1.0 - (j / trace)
                            color_fade = tuple(int(c * alpha_) for c in light_blue)
                            mx = int(arr[0, 0, 0])
                            my = int(arr[0, 0, 1])
                            cv2.circle(court_img, (mx, my), 3, color_fade, 20)

                # court lines
                if kps_court[i] is not None:
                    h, w = img_res.shape[:2]
                    draw_court_keypoints_and_lines(img_res, kps_court[i], w, h)

                # bounces => up to current frame
                if inv_mat is not None:
                    for bf in bounces:
                        if bf <= i and ball_track[bf][0]:
                            bpx, bpy = ball_track[bf]
                            arr = cv2.perspectiveTransform(
                                np.array([[[bpx, bpy]]], dtype=np.float32), inv_mat
                            )
                            mx = int(arr[0, 0, 0])
                            my = int(arr[0, 0, 1])
                            cv2.circle(court_img, (mx, my), 10, bounce_color, 40)

                # players
                minimap = court_img.copy()

                # -- top players => Player 1
                for bbox, center_pt in persons_top[i]:
                    if bbox is not None and len(bbox) == 4:
                        x1, y1, x2, y2 = map(int, bbox)
                        # Draw the bounding box on the main frame
                        cv2.rectangle(img_res, (x1, y1), (x2, y2), box_color, 2)
                        cv2.putText(
                            img_res,
                            "Player 1",
                            (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            box_color,
                            2,
                        )
                        # For the minimap, use the bottom-center point from PersonDetector
                        if inv_mat is not None:
                            cx, cy = center_pt  # <--- bottom-center from PersonDetector
                            arr = cv2.perspectiveTransform(
                                np.array([[[cx, cy]]], dtype=np.float32), inv_mat
                            )
                            mx = int(arr[0, 0, 0])
                            my = int(arr[0, 0, 1])
                            if (
                                0 <= mx < minimap.shape[1]
                                and 0 <= my < minimap.shape[0]
                            ):
                                cv2.circle(minimap, (mx, my), 48, box_color, -1)

                # -- bottom players => Player 2
                for bbox, center_pt in persons_bottom[i]:
                    if bbox is not None and len(bbox) == 4:
                        x1, y1, x2, y2 = map(int, bbox)
                        # Draw the bounding box on the main frame
                        cv2.rectangle(img_res, (x1, y1), (x2, y2), box_color, 2)
                        cv2.putText(
                            img_res,
                            "Player 2",
                            (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            box_color,
                            2,
                        )
                        # For the minimap, use the bottom-center point from PersonDetector
                        if inv_mat is not None:
                            cx, cy = center_pt
                            arr = cv2.perspectiveTransform(
                                np.array([[[cx, cy]]], dtype=np.float32), inv_mat
                            )
                            mx = int(arr[0, 0, 0])
                            my = int(arr[0, 0, 1])
                            if (
                                0 <= mx < minimap.shape[1]
                                and 0 <= my < minimap.shape[0]
                            ):
                                cv2.circle(minimap, (mx, my), 48, box_color, -1)

                # Store the minimap frame
                minimap_frames.append(
                    cv2.resize(minimap, (width_minimap, height_minimap))
                )

                # place minimap in main frame as before
                H, W = img_res.shape[:2]
                minimap_resized = cv2.resize(minimap, (width_minimap, height_minimap))
                img_res[0:height_minimap, 0:width_minimap] = minimap_resized

                imgs_res.append(img_res)
        else:
            # If the scene homography is mostly invalid, add blank minimaps
            blank_minimap = np.zeros((height_minimap, width_minimap, 3), dtype=np.uint8)
            minimap_frames.extend([blank_minimap] * (end_idx - start_idx))
            imgs_res += frames[start_idx:end_idx]

    return imgs_res, minimap_frames


def write(imgs_res, fps, output_path):
    if not imgs_res:
        print("No frames, skipping write.")
        return
    H, W = imgs_res[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (W, H))
    for frame in imgs_res:
        out.write(frame)
    out.release()
    print(f"[write] Finished writing {output_path}")


if __name__ == "__main__":
    path_ball_track_model = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/models/ball_detection_weights/tracknet_weights.pt"
    path_court_model = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/models/court_detection_weights/model_tennis_court_det.pt"
    path_bounce_model = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/models/bounce_detection_weights/bounce_detection_weights.cbm"
    path_input_video = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/dataset/game2/game2_1280x720.mp4"
    path_intermediate_video = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/game2_edited.mp4"
    path_output_video = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/game2_output.mp4"

    path_output_bounce_heatmap = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/heatmap/game2_bounce_heatmap.png"
    path_output_player_heatmap = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/heatmap/game2_player_heatmap.png"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    start_time = time.time()

    # 1) Scale to 720p if needed
    final_input = ensure_720p(path_input_video, path_intermediate_video)

    # 2) Read frames
    frames, fps = read_video(final_input)
    print(f"Loaded {len(frames)} frames at {fps} fps")

    # 3) Scene detection
    video_manager = VideoManager([final_input])
    stats_manager = StatsManager()
    scene_manager = SceneManager(stats_manager)
    scene_manager.add_detector(ContentDetector())
    base_timecode = video_manager.get_base_timecode()

    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list(base_timecode)
    video_manager.release()

    if not scene_list:
        scene_list = [(base_timecode, base_timecode + len(frames) / fps)]
    scenes = []
    for sc in scene_list:
        start_f = sc[0].frame_num
        end_f = sc[1].frame_num
        scenes.append((start_f, end_f))

    # 4) Ball detection
    ball_detector = BallDetector(path_ball_track_model, device)
    ball_track = ball_detector.infer_model(frames)

    # 5) Court detection
    court_detector = CourtDetectorNet(path_court_model, device)
    homography_matrices, kps_court = court_detector.infer_model(frames)

    # 6) Person detection
    person_detector = PersonDetector(device)
    persons_top, persons_bottom = person_detector.track_players(
        frames, homography_matrices, filter_players=False
    )

    # 7) Bounce detection
    bounce_detector = BounceDetector(path_bounce_model)
    x_ball = [bp[0] for bp in ball_track]
    y_ball = [bp[1] for bp in ball_track]
    bounces = bounce_detector.predict(x_ball, y_ball)

    # 8) Compose frames & write final videos
    print("Composing final frames...")
    imgs_res, minimap_frames = main(
        frames,
        scenes,
        bounces,
        ball_track,
        homography_matrices,
        kps_court,
        persons_top,
        persons_bottom,
        draw_trace=True,
        trace=7,
    )

    # Write main video
    write(imgs_res, fps, path_output_video)

    # Write minimap video
    path_minimap_video = "/content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/game2_minimap.mp4"
    write(minimap_frames, fps, path_minimap_video)

    # 9) Generate separate heatmaps for bounces/players
    generate_minimap_heatmaps(
        homography_matrices=homography_matrices,
        ball_track=ball_track,
        bounces=bounces,
        persons_top=persons_top,
        persons_bottom=persons_bottom,
        output_bounce_heatmap=path_output_bounce_heatmap,
        output_player_heatmap=path_output_player_heatmap,
        blur_ksize=41,
        alpha=0.5,
    )

    total_time = time.time() - start_time
    print(f"Done, saved to {path_output_video}. Overall time: {total_time:.2f}s")


Original input: 1280x720, fps=30.00
Video is already 1280x720; using input directly.


ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 30.000 FPS, resolution: 1280 x 720
INFO:pyscenedetect:Detecting scenes...


Loaded 1001 frames at 30 fps


ERROR:pyscenedetect:`base_timecode` argument is deprecated and has no effect.
  self.model.load_state_dict(torch.load(path_model, map_location=device))

100%|██████████| 1001/1001 [01:22<00:00, 12.13it/s]
  warnings.warn(

  warnings.warn(msg)

100%|██████████| 1001/1001 [00:37<00:00, 26.44it/s]


Composing final frames...
[write] Finished writing /content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/game2_output.mp4
[write] Finished writing /content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/game2_minimap.mp4
Saved bounce heatmap to: /content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/heatmap/game2_bounce_heatmap.png
Saved player heatmap to: /content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/heatmap/game2_player_heatmap.png
Done, saved to /content/drive/MyDrive/ASA Tennis Bounds Project/models/court_detection_model/detectron2/post_processing/game2/game2_output.mp4. Overall time: 182.48s
